In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import http.client, urllib.request, urllib.parse, urllib.error, base64
# import config
import quickstart

In [2]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1jxSI_nq32-W9kPHKvQkzuNGJ5aIBD2j2' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: analysis_bbl.csv, ID: 1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G
Title: borobox_convex_hull.geojson, ID: 1SzBNj3Ba-v9MEwuCkZT9e9iAwVYUqx8r
Title: analysis_bbl_noacs.csv, ID: 1h-tDpBSslfzqzsJe1ny_ujDnr68RKry3
Title: analysis_box.csv, ID: 1Vzjvta98IG9Hg4SfAGcd8JD8FaF3IXwy


In [4]:
csv = drive.CreateFile({'id': '1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G'})
csv.GetContentFile('analysis_bbl.csv')  

df = pd.read_csv('analysis_bbl.csv')

In [5]:
csv = df[["vacate_ind","hpd_b_c_vio", "bldgarea", "unitstotal", "pop65plus", "unitsres", "pctnhblack", "hpd_fire_vio"]]
csv.shape[0]
csv = csv.dropna()
csv.shape[0]

765026

In [ ]:
# consider bootstrapping the svms
# also just run it on a smaller subset

In [6]:
X = csv[["hpd_b_c_vio", "bldgarea", "unitstotal", "pop65plus", "unitsres", "pctnhblack", "hpd_fire_vio"]].head(200)
Y = csv[["vacate_ind"]].head(200)

In [7]:
# type(csv["vacate_ind"])
# i = 0
# for col in csv.loc[:, :"vacate_ind":].columns:
#     i+=1
#     print(col, i)
# Xdata = csv.drop(["vacate_ind"], axis=1)
from sklearn import preprocessing
X_scaled = preprocessing.scale(X)
X=X_scaled
print(pd.DataFrame(X).head())

          0         1         2         3         4         5         6
0 -0.347636 -0.348367 -0.365778  0.403473 -0.367491 -0.403473 -0.123404
1 -0.034274 -0.332229 -0.298969  0.403473 -0.297141 -0.403473 -0.123404
2 -0.347636  1.117746  0.686459  0.403473  0.740522 -0.403473 -0.123404
3 -0.347636 -0.170086 -0.048436  0.403473 -0.033328 -0.403473 -0.123404
4 -0.347636 -0.159247 -0.265565  0.403473 -0.261966 -0.403473 -0.123404


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=90)

In [9]:
print(len(X_train), len(Y_train))

134 134


In [10]:
from sklearn import svm
clf = svm.SVC(kernel='linear',C=1)

In [11]:
clf.fit(X_train, Y_train["vacate_ind"])
correct=1.0*(clf.predict(X_test) == np.asarray(Y_test).reshape(1,X_test.shape[0])).sum()/len(Y_test)
print("Out of sample, linear model successfully predicts {} percent of the data".format((correct)))

Out of sample, linear model successfully predicts 0.9848484848484849 percent of the data
